In [455]:
import pandas as pd
import numpy as np
from scipy import optimize
import sys
sys.path.append('../../')
from PortfolioAnalysis import Backtest

In [128]:
def inv(df):
    return pd.DataFrame(np.linalg.pinv(df.values), df.columns, df.index)

In [507]:
class BL:
    '''
    Black-Litterman is used to get the final combination weight
    Input:
    
    
    
    '''
    def __init__(self, df_date, df_history, df_fr, market_view, sub_view):
        self.df_date = df_date
        self.df_history = df_history
        self.df_fr = df_fr

        # market_view为风险评价模型观点矩阵
        # sub_view为宏观数据算出的主观观点矩阵
        self.market_view = market_view
        self.sub_view = sub_view
        
        # 权重
        ay_weight = self.market_view.iloc[[0]]
        
        # 期望收益率平均值
        df_r = np.log(df_history/df_history.shift(1))
        market_r = (self.market_view*np.array(df_r)).dropna()
        E_r = market_r.sum().sum()
        
        # 收益率标准差
        market_sigma = np.std(market_r.sum(axis=1))
        
        # 无风险利率平均值
        mean_fr = df_fr.mean()
        
        # 风险厌恶系数
        Lambda = ((E_r-mean_fr)/(market_sigma))[0]
        self.Lambda = Lambda
        
        # 资产收益率协方差矩阵(n*n)
        market_cov = df_r.corr() 
        
        # 先验收益率（n*1） = Lambda*协方差矩阵（n*n）*权重（n*1）
        Sigma = Lambda * np.dot(market_cov , ay_weight.T)
        self.Sigma = Sigma
        
        # 观点收益Q
        sub_view = self.sub_view
        Q = sub_view[sub_view.columns[len(sub_view.columns)-1]] .sum()
        
        # 观点信心水平LC默认为80%
        LC = 0.8
        
        # CF
        P = (self.sub_view.sum(axis=0))[:-1]
        P = pd.DataFrame(P).T
        CF = np.dot(np.dot(P,market_cov),P.T)[0]
        
        # 观点误差矩阵
        Omega = pd.DataFrame([CF/LC])
        
        # 投资者信心程度，默认为1
        tao = 1
        self.tao = tao
        
        # 后验收益率 E(R)
        E_R = np.dot(inv(inv(tao*market_cov)+np.dot(np.dot(P.T,inv(Omega)),P)),\
                         (np.dot(inv(tao*market_cov),self.Sigma)+(np.dot(np.dot(P.T,inv(Omega)),Q))))
        self.E_R = E_R
        
        # 新收益率协方差矩阵
        new_cov = inv(inv(tao*market_cov)+np.dot(np.dot(P.T,inv(Omega)),P))
       
        # 新的权重
        weight_new = inv(Lambda*new_cov).dot(E_R).T
        for i in range(62):
            weight_new = weight_new.append(weight_new.iloc[0]) 
        weight_new.index = self.df_date
        self.weight_new = weight_new
        
    
    def print_E_R(self):
        return self.E_R
    
    def print_weight(self):
        return self.weight_new
        

In [492]:
market_view = pd.read_csv("../Risk_Parity_Model/view.csv",index_col = 0)
sub_view = pd.read_csv("../Subjective_View_Setting/Subjective_View.csv",index_col = 0)
df_history = pd.read_csv("../../Data/Data_outsample.csv",index_col = 0)[:-28]

In [493]:
market_view = market_view[0:63]
sub_view = sub_view[0:63]
df_history = df_history[0:63]

In [494]:
df_date = df_history.index

In [495]:
len(df_date)

63

In [496]:
df_fr = pd.DataFrame(np.repeat(0,63))

In [508]:
xxx = BL(df_date, df_history, df_fr, market_view, sub_view)

In [509]:
xxx.print_weight()

,上证指数,沪深300,能源指数,工业指数,医疗保健指数,公用事业指数,中债-信用债总财富(总值)指数,中债-企业债总财富(总值)指数,中债-国开行债券总财富(总值)指数,南华农产品指数,黄金指数,日常消费指数,材料指数,电信服务指数,恒生综指
时间,,,,,,,,,,,,,,,
2016-03-29,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-03-30,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-03-31,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-04-01,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-04-05,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-23,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-06-24,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-06-27,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085


In [513]:
Backtest(df_date, df_history,df_fr, xxx.print_weight(),'D')

TypeError: 'module' object is not callable

In [502]:
ay_weight = market_view.iloc[[0]]

df_r = np.log(df_history/df_history.shift(1))
market_r = (market_view*np.array(df_r)).dropna()
E_r = market_r.sum().sum()

market_sigma = np.std(market_r.sum(axis=1))
mean_fr = df_fr.mean()
Lambda = ((E_r-mean_fr)/(market_sigma))[0]
market_cov = df_r.corr() 
Sigma = Lambda * np.dot(market_cov , ay_weight.T)
Q = sub_view[sub_view.columns[len(sub_view.columns)-1]] .sum()
LC = 0.8
P = (sub_view.sum(axis=0))[:-1]
P = pd.DataFrame(P).T
CF = np.dot(np.dot(P,market_cov),P.T)[0]
Omega = pd.DataFrame([CF/LC])
tao = 1
E_R = np.dot(inv(inv(tao*market_cov)+np.dot(np.dot(P.T,inv(Omega)),P)),\
                         (np.dot(inv(tao*market_cov),Sigma)+(np.dot(np.dot(P.T,inv(Omega)),Q))))
new_cov = inv(inv(tao*market_cov)+np.dot(np.dot(P.T,inv(Omega)),P))

In [503]:
Lambda

11.006243838685858

In [504]:
E_R

array([[ 2.29462977],
       [ 2.2194164 ],
       [ 1.37142891],
       [ 2.54117279],
       [ 2.77899004],
       [ 2.38428939],
       [ 5.62706643],
       [ 5.51255322],
       [ 5.67335292],
       [ 0.02834087],
       [ 1.01854735],
       [ 2.71237084],
       [ 2.34021213],
       [ 3.08648775],
       [-0.1352731 ]])

In [505]:
weight_new = inv(Lambda*new_cov).dot(E_R).T

In [506]:
weight_new.iloc[0]

上证指数                 0.008124
沪深300                0.007894
能源指数                 0.008760
工业指数                 0.006748
医疗保健指数               0.008260
公用事业指数               0.006261
中债-信用债总财富(总值)指数      0.287433
中债-企业债总财富(总值)指数      0.289289
中债-国开行债券总财富(总值)指数    0.288357
南华农产品指数              0.040869
黄金指数                 0.006287
日常消费指数               0.007818
材料指数                 0.007137
电信服务指数               0.007688
恒生综指                 0.015085
Name: 0, dtype: float64

In [499]:
weight_new.index=df_date

In [500]:
weight_new

,上证指数,沪深300,能源指数,工业指数,医疗保健指数,公用事业指数,中债-信用债总财富(总值)指数,中债-企业债总财富(总值)指数,中债-国开行债券总财富(总值)指数,南华农产品指数,黄金指数,日常消费指数,材料指数,电信服务指数,恒生综指
时间,,,,,,,,,,,,,,,
2016-03-29,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-03-30,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-03-31,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-04-01,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-04-05,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-23,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-06-24,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085
2016-06-27,0.008124,0.007894,0.00876,0.006748,0.00826,0.006261,0.287433,0.289289,0.288357,0.040869,0.006287,0.007818,0.007137,0.007688,0.015085


In [108]:

        # CF
        P = sub_view.sum(axis=0)
        CF = np.dot(np.dot(P,market_cov),P.T)
        
        # 观点误差矩阵
        Omega = CF/LC
        
        # 投资者信心程度，默认为1
        tao = 1
        self.tao = tao
        
        # 后验收益率 E(R)
        E_R = np.dot(inv(inv(tao*market_cov)+np.dot(np.dot(P.T,inv(Omega)),P)),\
                         (np.dot(inv(tao*market_cov),self.Sigma)+(np.dot(np.dot(P.T,inv(Omega)),P))))
        self.E_R = E_R
        
        # 新收益率协方差矩阵
        new_cov = inv(inv(tao*market_cov)+np.dot(np.dot(P.T,inv(Omega)),P))

上证指数                  48.634054
沪深300                 97.737535
能源指数                -143.684916
工业指数                  35.478149
医疗保健指数                75.780662
公用事业指数                89.881690
中债-信用债总财富(总值)指数      312.688967
中债-企业债总财富(总值)指数      353.316756
中债-国开行债券总财富(总值)指数    150.648543
南华农产品指数              -62.909313
黄金指数                  52.053151
日常消费指数               100.235902
材料指数                 -12.876276
电信服务指数               -33.294670
恒生综指                  70.309767
Q                     46.247229
dtype: float64

In [109]:
P = sub_view[sub_view.columns[len(sub_view.columns)-1]]

In [125]:
np.array([3])

array([3])

In [122]:
np.dot(P,np.array([3])_inv)

SyntaxError: invalid syntax (<ipython-input-122-3b2d7f1282d4>, line 1)

In [120]:
np.dot(P,P.T)

38.18238141476492

In [127]:
P.values

array([-0.14697862, -0.14697862, -0.14697862, ...,  0.00063078,
        0.00063078,  0.00063078])

In [77]:
df_view1.iloc[[0]]

,上证指数,沪深300,能源指数,工业指数,医疗保健指数,公用事业指数,中债-信用债总财富(总值)指数,中债-企业债总财富(总值)指数,中债-国开行债券总财富(总值)指数,南华农产品指数,黄金指数,日常消费指数,材料指数,电信服务指数,恒生综指
时间,,,,,,,,,,,,,,,
2016-03-29,0.008272,0.007829,0.00734,0.006385,0.007445,0.006861,0.289775,0.291762,0.289649,0.040803,0.007039,0.007992,0.006117,0.007013,0.015719


In [75]:
start = 0
df_view1 df_view1.index[start:start+6*252][0]


'2016-03-29'

In [72]:
df_date = df_view1.index[start:start+6*252]

In [94]:
np.array([[1],[2],[3]])

array([[1],
       [2],
       [3]])

In [92]:
np.array([1,2,3])

array([1, 2, 3])

In [95]:
np.dot(np.array([1,2,3]),np.array([[1],[2],[3]]))

array([14])

In [90]:
np.array([1,2,3])*df_return_each

,0,1,2
0,1,2,6
1,0,2,0
2,2,4,0
3,3,0,15
4,5,0,18
5,0,4,-21


In [38]:
df_date = pd.DataFrame({"date":["2020/01/01","2020/01/02","2020/01/03","2020/01/04","2020/01/05","2020/01/06"]})
df_view = pd.DataFrame({"v1":[1,0,1,1,1,0],"v2":[1,1,2,0,0,1],"v3":[1,0,0,1,1,-1]})
df_netvalue = pd.DataFrame({"p1":[1,2,2,3,5,6],"p2":[1,1,1,1,1,2],"p3":[2,3,4,5,6,7]})
initial_asset = 100

In [34]:
pd.concat([df_date,df_view,df_history], axis=1)

,date,v1,v2,v3,p1,p2,p3
0,2020/01/01,1,1,1,1,1,2
1,2020/01/02,0,1,0,2,1,3
2,2020/01/03,1,2,0,2,1,4
3,2020/01/04,1,0,1,3,1,5
4,2020/01/05,1,0,1,5,1,6
5,2020/01/06,0,1,-1,6,2,7


In [39]:
ay_view = np.array(df_view)
ay_hisotry = np.array(df_netvalue)
df_return_each = pd.DataFrame(ay_view*ay_netvalue)
df_return = df_price_each.sum(axis=1)

In [86]:
sum(sum(np.array(df_return_each)*np.array([0.1,0.2,0])))

2.3000000000000003

In [81]:
df_return_each

,0,1,2
0,1,1,2
1,0,1,0
2,2,2,0
3,3,0,5
4,5,0,6
5,0,2,-7


In [42]:
pd.DataFrame(ay_view*ay_netvalue)

,0,1,2
0,1,1,2
1,0,1,0
2,2,2,0
3,3,0,5
4,5,0,6
5,0,2,-7


In [26]:
# the price of asset when position changes
ay_view_close = np.array(df_view.shift(1))*np.repeat(ay_view_cg,3).reshape(6,3) + np.array(df_view.shift(0))*np.repeat(ay_view_cg==0,3).reshape(6,3)
df_price_each_close = pd.DataFrame(ay_view_close*ay_history)
df_price_each_close.iloc[0,:] = df_price_each.iloc[0,:]
ay_price_each_close = np.array(df_price_each_close)

In [27]:
ay_price_each_close

array([[1., 1., 2.],
       [2., 1., 3.],
       [0., 1., 0.],
       [3., 2., 0.],
       [5., 0., 6.],
       [6., 0., 7.]])